In [ ]:
from __future__ import print_function

In [ ]:
from nbtemplate import display_header, display_codetoggle, get_path
display_header('SelectRowlandParameters.ipynb', status='in development')

In [ ]:
import os
import sys
from glob import glob
import functools

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import astropy.units as u

from marxs.analysis import gratings as anagrat

%matplotlib inline

In [ ]:
filelist = glob(os.path.join(get_path('grid2designtorus'), '*.fits'))

In [ ]:
filelist.sort()  # for display purposes. Also, makes the order more predictable
len(filelist)

In [ ]:
def zero_pos(p, coord='circ_phi'):
    ind = (p['order'] == 0) & np.isfinite(p[coord])
    return np.mean(p[coord][ind])

def angle_covered_by_CCDs(p, n_ccds=8):
    return n_ccds * 25. / p.meta['CIRCLE_R']

In [ ]:
def add_phifolded(p):
    # calculate which angles can be covered
    phi_0 = zero_pos(p)

    # phi at the point excatly half-way between the two channels, the "middle"
    phi_m = phi_0 - np.arcsin(p.meta['D_CHAN'] / 2 / p.meta['CIRCLE_R'])

    # make new column "distance from phi_m"
    p['phi_folded'] = np.abs(p['circ_phi'] - phi_m)

In [ ]:
def plot_phi_hist(ax, p):
    p_order = p['order', 'phi_folded', 'probability'].group_by('order')
    bins = np.linspace(0, .2, 41)
    sl = slice(-1, 2, -1)
    phis = [g['phi_folded'] for g in p_order.groups]
    probs = [g['probability'] for g in p_order.groups]
    ax.hist(phis[sl], weights=probs[sl], bins=bins, stacked=True, 
            label=[np.int(np.abs(k[0])) for k in p_order.groups.keys[sl]], rwidth=1, ec='none')
    ax.legend(title='Order')
    ax.set_xlabel('dispersion angle [rad]')
    ax.set_ylabel('# photons')

In [ ]:
def aeff(p):
    p_wave = p['wave', 'probability'].group_by('wave')
    aeff = p_wave['wave', 'probability'].groups.aggregate(np.sum)
    aeff['probability'] *= p.meta['A_GEOM'] * 4  / p.meta['N_PHOT']
    aeff.rename_column('probability', 'area')
    aeff['area'].unit = u.cm**2
    return aeff
    
def plot_aeff(ax, p, **kwargs):
    aeffd = aeff(p[p['order'] != 0])
    label = kwargs.pop('label', '__no_legend__')
    ax.plot(aeffd['wave'], aeffd['area'], label=label, **kwargs)
    aeff0 = aeff(p[p['order'] == 0])
    ax.plot(aeff0['wave'], aeff0['area'], ':', **kwargs)

In [ ]:
# Need to find function that selectes the "optimal" detector position. 
# That clearly depends on the sceince objective, so I might in fact try different functions.
# Here is a first attempt:

def ccd8zeroorder(p, phi_0):
    ang8 = angle_covered_by_CCDs(p, n_ccds=8) 
    binwidth = angle_covered_by_CCDs(p, n_ccds=0.1) 
    # Look at region +- 8 CCDS fro mthe zeros order, because we definitely want that in the range.
    # Don't go +/- 8 CCDs, but a little less, so zeroth order is never exactly on the edge of detector
    bins = np.arange(phi_0 - (ang8 - binwidth), phi_0 + (ang8 + binwidth), binwidth)
    # This is where more contraints (e.g. optimize only the aeff for O VII) would happen
    hist, bin_edges = np.histogram(p['phi_folded'], weights=p['probability'], bins=bins)
    signal = np.cumsum(hist)
    signal8 = signal[80:] - signal[:-80]
    return bins[np.argmax(signal8)]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plot_aeff(ax, pclean, color='b')
plot_aeff(ax, pdet16, color='g')
plot_aeff(ax, pdet16o7, color='r')

In [ ]:
# Filter by probability because the resolvingpower_from_photonlist currently does not take that into account.
# Probabilites are all very similar and pclean already has the 0 prob photons removed
orders = np.arange(-12, 0.1)

def calc_resolution(photons, orders):
    '''Calculate the resolving power from a simulation result
    
    Parameters
    ----------
    photons : photon list
    orders : np.array of shape (M,)
    
    Returns
    -------
    wave : np.array of shape (N,)
        wavelength array
    resolvingpower : np.array of shape (N,)
        Resolving power averages of all dispersed orders
    res_out : np.array of shape (M, N)
        Resolving power per order
    probs : np.array of shape (M, N)
        Weighting factor for the power received in every order
    '''
    p_wave = photons['energy', 'probability', 'circ_phi', 'order'].group_by('energy')
    res_out = np.zeros((len(p_wave.groups), len(orders)))
    prob_out = np.zeros_like(res_out)
    for i, group in enumerate(p_wave.groups):
        res, pos, std = anagrat.resolvingpower_from_photonlist(group, orders, col='circ_phi')
        res_out[i, :] = res
        # Now get how important every order is for order-weighted plots
        for j, o in enumerate(orders):
            prob_out[i, j] = (group['probability'][group['order'] == o]).sum()
    # Normalize prob_out
    probs = prob_out / prob_out.sum(axis=1)[:, None]
    ind = (orders != 0)
    resolvingpower = np.nansum(res_out[:, ind] * probs[:, ind], axis=1)
    wave = p_wave.groups.keys['energy'].to(u.Angstrom, equivalencies=u.spectral())
    return wave, resolvingpower, res_out, probs

In [ ]:
wave, res, res_out, probs = calc_resolution(pclean, orders)
out = plt.plot(wave, res_out)
out = plt.plot(wave, res, 'b', lw=4)

wave, res, res_out, probs = calc_resolution(pdet16, orders)
out = plt.plot(wave, res, 'g', lw=4)

wave, res, res_out, probs = calc_resolution(pdet16o7, orders)
out = plt.plot(wave, res, 'r', lw=4)


In [ ]:
def plot_everything(p):
    add_phifolded(p)
    p['wave'] = p['energy'].to(u.Angstrom, equivalencies=u.spectral())
    # make clean sample of photons
    ind = np.isfinite(p['circ_phi']) & np.isfinite(p['order']) & (p['probability'] > 0)
    pclean = p[ind]
    
    pcleandisp = pclean[pclean['order'] < 0]

    phistart = ccd8zeroorder(pcleandisp, np.abs(phi_0 - phi_m))
    pdet16 = pclean[(pclean['phi_folded'] > phistart) & (pclean['phi_folded'] < (phistart + angle_covered_by_CCDs(p)))]

    po7 = pcleandisp[(pcleandisp['wave'] > 19.9) & (pcleandisp['wave'] < 22.5)]
    phistart = ccd8zeroorder(po7, np.abs(phi_0 - phi_m))
    pdet16o7 = pclean[(pclean['phi_folded'] > phistart) & (pclean['phi_folded'] < (phistart + angle_covered_by_CCDs(p)))]
    
    plists = [pclean, pdet16, pdet16o7]
    colors = 'bgr'
    
    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(131)
    plot_phi_hist(ax1, pclean)
    
    ax2 = fig.add_subplot(132)
    for pl, c in zip(plists, colors):
        plot_aeff(ax2, pl, color=c)

    ax3 = fig.add_subplot(133)
    wave, res, res_out, probs = calc_resolution(pclean, orders)
    ax3.plot(wave, res_out)
    ax3.plot(wave, res, 'b', lw=4)

    wave, res, res_out, probs = calc_resolution(pdet16, orders)
    ax3.plot(wave, res, 'g', lw=4)

    wave, res, res_out, probs = calc_resolution(pdet16o7, orders)
    ax3.plot(wave, res, 'r', lw=4)
    
    return fig

In [ ]:
p = Table.read(filelist[-4])
fig = plot_everything(p)

In [ ]:
p = Table.read(filelist[30])
fig = plot_everything(p)